In [ ]:
from IPython.display import YouTubeVideo, Markdown, SVG
from functools import partial
YouTubeVideo_formato = partial(YouTubeVideo, modestbranding=1, disablekb=0,
                               width=640, height=360, autoplay=0, rel=0, showinfo=0)

display(Markdown(filename='../preamble.md'))

# Exploración y manipulación de datos con [*pandas*](https://pandas.pydata.org/) <a class="tocSkip"></a>

En la clase anterior aprendimos a operar DataFrames construidos a partir de estructuras de datos de Python

> Pero el caso más general de uso de pandas es la exploración y manipulación de una base de datos tabular que podría existir como un fichero en nuestro sistema o en un servidor remoto

En este clase veremos 
- como crear DataFrames a partir de distintas fuentes de datos
- funciones más avanzadas de manipulación de DataFrames

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

display("Versión de pandas "+pd.__version__)

## Importar datos tabulares a partir de archivos CSV 

Un archivo  **CSV** (Comma-Separated Values) es 

> una tabla en formato texto plano cuyas columnas están separadas por comas (u otro delimitador)

**Ejemplo**

Base de datos "Dow Jones Index" en formato CSV del [repositorio UCI](https://archive.ics.uci.edu/ml/datasets/Dow+Jones+Index)

La descargamos y observamos las 5 primeras lineas

In [ ]:
%%bash
wget -c https://archive.ics.uci.edu/ml/machine-learning-databases/00312/dow_jones_index.zip
unzip -o dow_jones_index.zip
head -5 dow_jones_index.data

El Dow Jones es un índice bursatil muy utilizado ya que refleja el comportamiento del mercado accionario norteamericano

Podemos ver que cada fila tiene un 
- identificador textual de la acción: `AA`
- una fecha de observación: `1/7/2011`
- un precio de apertura, máximo, mínimo y cierre para la fecha: `$15.82, $16.72, $15.78, $16.42`
- entre otros

Podemos notar algunos aspectos típicos de los archivos CSV
- La primera fila del archivo CSV contiene el *header*, es decir los nombres de las columnas
- Las columnas son de tipos distintos 

¿Qué tipos puedes identificar en el ejemplo anterior?

### Función `pd.read_csv` y atributo `to_csv()`

Leer un archivo CSV como DataFrame es directo usando la función (sólo algunos argumentos resaltados)

```python
pd.read_csv(
    filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]], # path completo al archivo CSV
    sep=',', # String o expresión regular que se usará para delimitar las columnas
    header='infer', # Puede ser un int (fila donde está el header) o una lista de de int's
    names=None, # Lista de strings con nombres de columnas (útil si el CSV no tiene header)
    index_col=None, # La columna que se usará como header
    usecols=None, # Lista: subconjunto de columnas que se desean importar (por defecto se importan todas)
    converters=None, # Se explica en detalle más adelante junto a otros argumentos de parsing
    parse_dates=None, # Se explica en detalle más adelante junto a otros argumentos de fecha
    ...
    )
```

También podemos crear un archivo CSV desde un DataFrame usando

```python
    df = pd.DataFrame(data)
    df.to_csv("mis_datos.csv")
```

- Esto  crea un archivo `mis_data.csv` en el directorio actual
- Por defecto guardara las nombres de columna como un header y usará "," como delimitador

### Análisis sintático o *parsing*

Los textos de un archivo CSV pueden representar 
- valores numéricos continuos
- valores numéricos discretos
- fechas
- coordenadas 
- moneda
- direcciones
- etiquetas de texto
- entre **muchos** otros

Los programas que importan un archivo CSV deben interpretar estos valores y convertirlos al formato más adecuado, por ejemplo
- flotante
- entero
- booleano
- string

> Se llama ***parser* o analizador sintático** al programa que analiza los textos y luego 
- filtra y/o completa los textos invalidos
- convierte los datos a un formato estándar

Pandas hace este proceso de forma automática y podemos hacer algunos ajustes usando ciertos argumentos disponibles en `read_csv`, por ejemplo 


```python
pd.read_csv(
    ...
    dtypes=None # Diccionario donde la llave es el nombre de la columna y el valor el tipo requerido
    na_values=None, # String o lista de strings con valores que serán reconocidos como NaN
    decimal='.', # String que se usará para reconocer el punto decimal
    comment=None, # String, todos las lineas que empiezen con este string serán ignoradas
    converters=None # Se explica a continuación
    ...
    )
```

> Si las opciones automáticas no son suficientes se puede hacer *parsing* en base a reglas manualmente creadas

El argumento `converters` recibe un diccionario con "reglas de parseo" como sigue

```python
    {'nombre de la columna 1': funcion_parseadora1, 
     'nombre de la columna 2': funcion_parseadora2,
     ...
    }
```

Donde `funcion_parseadoraX` puede ser una función explicita o anómina (lambda)



**Ejemplo**

Los datos de la columna de precio de apertura (open) de "dow_jones_index.data" están formateados como 

`'$15.84'`

Es decir un signo dolar seguido de un número real con punto decimal

Para *parsear* este valor debemos escribir una función que 
1. Elimine el signo dolar del string
1. Convierta el resto del string en flotante

Luego creamos el diccionario que entregaremos al argumento `converters`

```python
    def remove_dollar(text):
        # return float(x[1:]) # Elimina el primer caracter
        return float(x.strip("$")) # Elimina todos los $ del string
        
    parser = {'open': remove_dollar}
```

o con una función anónima

```python
    parser = {'open': lambda x: float(x.strip("$"))}
```


### Interpretación Fechas

Un dato textual muy usual en datos tabulares y series de tiempo son las fechas. Pero el formato de fecha puede variar considerablemente entre distintas bases de datos

> Pandas tiene un tipo denominado `Timestamp` el cual se puede construir con la función `pd.to_datetime()` a partir de un string 

Pandas identifica automaticamente fechas y horas en distintos formatos

**Ejemplo**

```python
>>> pd.to_datetime("1/5/2018") # Formato norteamericano Mes/Día/Año 
Timestamp('2018-01-05 00:00:00')

>>> pd.to_datetime("May/1/2018") # También se acepta un string para el mes
Timestamp('2018-05-01 00:00:00')

>>> pd.to_datetime("1st of May of 2018") # También se puede usar una frase "Día del Mes del Año"
Timestamp('2018-05-01 00:00:00')

>>> pd.to_datetime("2018") # Autocompletación por defecto para fechas incompletas
Timestamp('2018-01-01 00:00:00')

>>> pd.to_datetime("14:45") # Si usamos sólo la hora se usa la fecha actual
Timestamp('2020-06-12 14:45:00')

>>> pd.to_datetime("May/1/2018 14:45") # Timestamp completo
Timestamp('2018-05-01 14:45:00')
```

Podemos controlar el parseo de fechas en `read_csv` con los argumentos

```python
pd.read_csv(
    ...
    parse_dates=False # Booleano o lista con las columnas que deben ser interpretadas como fechas
    infer_datetime_format=False, # Inferir una función parseadora de forma automática
    dayfirst=False, # Formato día/mes/año o mes/día/año
    date_parser=None, # Función provista por el usuario que toma un string y retorna TimeStamp
    ...
    )
```

Las fechas/tiempos en formato *timestamp* pueden usarse como índices

> Esto nos permite recuperar rapidamente todos los eventos dentro de un intervalo de tiempo

**Ejercicio:**

1. Lea el archivo `dow_jones_index.data` con `pd.read_csv` con las opciones por defecto y estudie el DataFrame resultante
1. Corrija incrementalmente:
    1. Use un conversor para todas las columnas numéricas que empiezan con `$`
    1. Use el argumento `parse_dates` para parsear la columna date como un `Timestamp` de pandas

En cada paso verifique el tipo de las columnas con el atributo `dtypes`

Con su tabla adecuadamente formateada: 
> Retorne los valores de apertura (open), cierre (close), mínimo (low) y máximo (high) para las acciones de Alcoa Corp. (AA) entre el Marzo y Junio del 2011


In [ ]:
# Completa tu solución acá


In [ ]:
YouTubeVideo_formato('zrBQQQZeGXw')

## Importar datos tabulares a partir de archivos excel

Muchas empresas e instituciones manejan sus datos como hojas de cálculo o *spreadsheets* construidas usando software como Microsoft Excel, Openoffice/Libreoffice calc o Google spreadsheets

Pandas permite importar como dataframe hojas de cálculo en formatos `xls, xlsx, xlsm, xlsb, and odf` usando  la función [`read_excel`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html)

   
Muchos de los argumentos de `read_csv` están disponibles en `read_excel`, los nuevos argumentos son

```python
pd.read_excel(io, # string o path a la hoja de cálculo
              sheet_name=0, # Entero, string o lista, especifica la(s) hoja (s) que vamos a importar
              ...
             )
```

**Nota** 

Esta función depende adicionalmente de la librería de Python [xlrd](https://xlrd.readthedocs.io/en/latest/) que se puede instalar con `conda`:

    conda install xlrd
    
    
**Ejemplo**

Consideremos los siguientes datos del censo chileno de 2017 en formato Excel de donde importaremos datos de vivienda por comuna

Esto corresponde a la segunda hoja (`sheet_name=1`) y en particular las columnas de 1 a 20



In [ ]:
!wget -c http://www.censo2017.cl/wp-content/uploads/2017/12/Cantidad-de-Viviendas-por-Tipo.xlsx

In [ ]:
df = pd.read_excel("Cantidad-de-Viviendas-por-Tipo.xlsx", 
                   sheet_name=1, # Importamos la segunda hoja (vivienda)
                   usecols=list(range(1, 20)), # Importamos las columnas 1 a 20
                   header=1, # El header está en la segunda fila
                   skiprows=[2], # Eliminamos la fila 2 ya que es invalida
                   index_col='ORDEN' # Usaremos el orden de las comunas como índice
                  ).dropna() # Eliminamos las filas con NaN

display(df.head())

## Manipulando índices y multi-índices

Estudiando la tabla anterior notamos que tiene una estructura jerárquica

> REGION, PROVINCIA, COMUNA

Podemos representar este tipo de estructuras en pandas usando `MultiIndex` 

Para asignar un índice a un DataFrame que ya está creado podemos usar el atributo

```python
df.set_index(keys, # Una etiqueta o una lista de etiquetas que serán los nuevos índices
             drop=True, # Eliminar las columnas que pasarán a ser índices
             inplace=False, # Retornar un nuevo dataframe o modificar df
             ...
            )
```

- Si keys es una etiqueta crearemos un índice regular
- Si keys es una lista crearemos un `MultiIndex`

```python
>>> df.set_index(["NOMBRE REGIÓN", "NOMBRE PROVINCIA"], inplace=True) 
>>> # Se crea un MultiIndex de dos niveles de jerarquía en df
>>> df.index
MultiIndex([(                  'ARICA Y PARINACOTA',             'ARICA'),
            (                  'ARICA Y PARINACOTA',             'ARICA'),
            (                  'ARICA Y PARINACOTA',        'PARINACOTA'),
            ...
```

Si queremos que nuestro índice o multi-índice vuelva a convertirse en columna podemos usar el atributo

```python
df.reset_index(level = None, # Permite especificar cuantos niveles de índices se removeran
               drop: bool = False, # Si los índices se deben eliminar o agregar como columnas
               inplace: bool = False,  # Retornar un nuevo dataframe o modificar df
               ...
               )
```

Para recuperar un elemento de un DataFrame con `MultiIndex` podemos indexar usando una tupla especificando cada uno de los niveles de índices

```python
>>> df.loc[("LOS RÍOS", "VALDIVIA", "VALDIVIA")] # Nos retorna una fila
```

Si queremos recuperar un conjunto de elementos podemos usar

```python
>>> df.loc[("LOS RÍOS", "VALDIVIA")] # Retorna todas las comunas de la provincia de Valdivia
>>> df.loc[("LOS RÍOS")] # Retorna todas las comunas de región de los rios
```

Si queremos usar fancy indexing o slicing lo más recomendable es usar el objeto [`IndexSlice`](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.IndexSlice.html) para generar slices

```python
>>> idx = pd.IndexSlice
>>> df.loc[idx[:, :, ["VALDIVIA", "OSORNO"]], :] # Las comunas de Valdivia y Osorno
>>> df.loc[idx[:, ["LLANQUIHUE", "PALENA"], : ], :] # Las comunas de las provincias de Llanquihue y Palena
```




**Ejercicio**

- Asigne un MultiIndex al DataFrame de datos de vivienda del censo
    - Use como primer nivel la etiqueta "NOMBRE REGION"
    - Use como segundo nivel la etiqueta "NOMBRE PROVINCIA" 
    - Use como tercer nivel la etiqueta "NOMBRE COMUNA"
- Use `loc` para seleccionar
    - las comunas de la región de "LOS RÍOS"
    - las comunas de las provincias de "RANCO" y "OSORNO"
    - las comunas "VALDIVIA" y "FRUTILLAR"
- Selecciona las comunas de la provincia de "VALDIVIA" y usa una reducción suma para encontrar el número de viviendas totales de cada tipo

In [ ]:
# Completa tu solución acá


In [ ]:
YouTubeVideo_formato('bWjB4089EbA')

## Agrupamiento en DataFrames usando Groupby

Digamos que queremos obtener los totales de todos los tipos de vivienda a nivel de provincia

Si asignamos "NOMBRE PROVINCIA" como índice podríamos usar

```python
result = []
for provincia in df.index.unique():    
    sub_df = df.loc[provincia, col_mask]
    if sub_df.ndim>1:    
        result.append(df.loc[provincia, col_mask].sum())
    else: # No hacer reducción suma si la provincia tiene una sola comuna
        result.append(df.loc[provincia, col_mask])
pd.DataFrame(result, columns=col_mask, index=df.index.unique())
```

que obtiene el resultado que queremos pero es bastante engorroso

> Esta operación condicionada por una llave se conoce como **agrupamiento**. A continuación veremos como hacer agrupamiento de forma simple y eficiente con pandas

Los DataFrame tienen un atributo llamado [`groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) 

Este atributo permite implementar una [secuencia](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) de tres pasos como la que sigue



Donde
- *split*: Divide los datos según una **llave**
- *apply*: Realiza una función sobre cada grupo
- *combine*: Mezcla el resultado en un nuevo dataframe donde la **llave** se convierte en el índice

Los argumentos básicos de `groupby` son

```python
df.groupby(by=None, # Columna o lista de columnas con se hace el split
           axis=0, # Dividir en filas (0) o en columnas (1)
           as_index: bool = True, # Retornar las etiquetas de cada grupo como índice
           sort: bool = True, # Retornar las etiquetas de grupo ordenadas alfabeticamente
           ...
          )    
```

Groupby actua como un iterador

```python
for (region, sub_df) in df.groupby('NOMBRE REGIÓN'):
    display(region, # La etiqueta
            sub_df  # El dataframe con las filas que comparten esa etiqueta
           )
```

La función que se ejecuta a cada grupo en el paso *apply* es un atributo de `groupby`




### Agrupamiento y reducción

El atributo se llama `aggregate` y la sintaxis básica es

```python
    df.groupby(by=llave).aggregate(funcion1) 
    # Se aplica funcion1 a todas las columnas
    df.groupby(by=llave).aggregate([funcion1, funcion2, ...]) 
    # Se aplican todas las funciones a todos las columnas
    df.groupby(by=llave).aggregate({columna1: funcion1, columna2: funcion2}) 
    # Se aplican funcion1 a columna1 y funcion2 a columna2, respectivamente
```

Las funciones debe entregar un número por cada columna del grupo

Se usa para hacer resumenes, por ejemplo sumas, promedios o varianzas

**Ejemplo: Agrupamiento con reducción**

Podemos encontrar los totales por provincia en una sola linea usando

```python
>>> df.groupby(by="NOMBRE PROVINCIA", sort=False).aggregate(np.sum)
>>> # df.groupby(by="NOMBRE PROVINCIA", sort=False).sum() # alternativa
```



### Agrupamiento y filtro

El atributo se llama `filter` y la sintaxis básica es

```python
    df.groupby(by=llave).filter(funcion)
```

La función debe retornar `True` o `False`

Se recupera un nuevo DataFrame con todos los grupos que pasaron el filtro

Se usa para eliminar grupos de filas (drop)


### Agrupamiento y transformación

El atributo se llama `transform` y la sintaxis básica es

```python
    df.groupby(by=llave).transform(funcion)
```

La función 
- debe retornar un dataframe con la misma dimensión y tamaño que el original
- se aplica columna a columna
- puede ser explicita o anónima

Se puede usar para reescalar o normalizar los valores de las columnas a nivel de grupo

**Ejercicio**

Considere las columnas de "viviendas particulares ocupadas con moradores presentes" ($V_1$) y "viviendas particulares ocupadas con moradores ausentes" ($V_2$)

1. Cree un MultiIndex equivalente al del ejercicio anterior
1. Realice una reducción promedio y desviación estándar de cada región
1. Use un filtro para encontrar las comunas "más responsables", es decir aquellas donde $\frac{V_1}{V_1 + V_2} > 0.98$
1. Use una transformación para describir las columnas $V_1$ y $V_2$ como porcentajes a nivel regional

In [ ]:
# Completa tu solución acá


In [ ]:
YouTubeVideo_formato('BopVrkZKNtw')

## Lectura de bases de datos SQL


Pandas es capaz de conectar y hacer consultas en lenguaje SQL a una base de datos externa y retornar el resultado como un DataFrame usando la función [`read_sql_query`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) 

```python
pd.read_sql_query(sql, # Consulta SQL en formato string
                  con, # dirección a la base de datos o objeto de conexión
                  index_col=None, # Selecciona la columna que actuara como índice del DataFrame
                  parse_dates=None, # Igual que read_csv y read_excel
                  ...
                 )
```

También se puede usar el atributo

```python
df.to_sql(name, # string: el nombre de la tabla
          con, # Engine con conexión
          if_exists: str = 'fail', # Que hacer si la tabla ya existe: fail, replace, append
          index: bool = True, # Escribir el índice del dataframe como columna
          ...
         )
```

**Qué es SQL?**

Structured Query Languaje (SQL) es un lenguaje estándar ampliamente usado para consultar, crear, modificar y eliminar bases de datos relacionales. 

**Qué es una base de datos relacional?**

Es un tipo de base de datos organizada como múltiples tablas. Por ejemplo


|id_cliente | nombre | apellido |
|----|----|----|
|1| Pablo | Huijse |
|2| Luis | Alvarez |
|3| Cristobal | Navarro |
|  | CLIENTES |  |

|id_orden | platanos | manzanas | id_cliente |
|----|----|----| ---- |
|1| 0 | 5 | 1 |
|2| 2 | 2 | 3 |
|3| 3 | 1 | 1 |
|  | ORDENES |  | | 

- Las filas se llaman entidades y las columnas atributos
- Cada tabla tiene una lalve primaria: id_orden e id_cliente
- La tabla ORDENES **está relacionada** a la tabla CLIENTES con la llave foranea: id_clientes
- Las tablas no pueden tener el mismo nombre 






**Donde corre la base de datos relacional?**

La base de datos relacional corre en un sistema de manejo 

Algunos ejemplos populares son MySQL, PostgreSQL y SQLite3


**Ejemplo básico de una consulta SQL**

SQL es un lenguaje de alto nivel. Algunos comandos comunes son

- `SELECT`: recuperar un subconjunto de la tabla
- `INSERT`: insertar datos en una tabla
- `UPDATE`: actualizar datos en una tabla
- `DELETE`: eliminar datos de la tabla

La tabla que se quiere manipular se selecciona con el keyword `FROM`

Se agregan condiciones usando el keyword `WHERE`

Se puede usar `*` como alias para "todas las columnas"

Por ejemplo
```SQL
    SELECT A, B, C FROM mi_tabla WHERE D > 1
```

Esto recupera las valores de las columnas A, B y C que tegan un valor de la columna D mayor que 1 a partir de la tabla "mi_tabla" 


**Ejercicio**

Escriba y prueba una consulta SQL que recupere un DataFrame con las columnas "Viviendas Particulares Ocupadas con Moradores Presentes" y "NOMBRE COMUNA" para la provincia de Valdivia

Indicación: Cuando escriba su consulta encierre los nombres de las columnas con paréntesis cuadrados. Esto es necesario cuando los nombres contienen espacios en blanco

In [ ]:
#Primero vamos a crear una base de datos e insertar un DataFrame como tabla

import sqlite3  # SQLite3 es parte de la librería estándar de Python

# Creamos una base de datos persistente
with sqlite3.connect('censo.db') as conn:

    df.to_sql("censo_viviendas", # Insertamos una tabla llamada censo_viviendos
              conn, # Usamos el objeto conexión que acabos de crear
              if_exists='replace', 
              index=False)

In [ ]:
# Escribe tu solución aquí


In [ ]:
YouTubeVideo_formato('VsoZ1oh8Az8')

**Notas sobre SQLite**

- sqlite permite conectar a una base de datos local: RAM, disco, o disco externo montado
- sqlite no está diseñado para soportar múltiples usuarios conectados a una misma base de datos
- Otras alternativas: [SQL Alchemy](https://www.sqlalchemy.org/), [PostgreSQL+Python](http://initd.org/psycopg/), [Peewee](http://docs.peewee-orm.com/en/latest/)

## Guardar y leer una tabla  otros formatos


### JSON

Podemos usar el atributo [`to_json`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html) para convertir un dataframe a este formato

```python
df.to_json(
    path_or_buf = None, # Ubicación en disco
    orient = None, # Indica el formato del string JSON
    ...
    )
```

Por ejemplo

```python
>>> df.to_json("pandas.json", orient='table')
```

crea un string pandas.json en el directorio actual

Luego la función `read_json`

```python
>>> df = pd.read_json("pandas.json", orient='table')
```

regenera el DataFrame que teniamos

### HDF5

Podemos usar el atributo `to_hdf` para convertir nuestra tabla a formato HDF5

```python
df.to_hdf(path_or_buf, # Path completo con nombre de archivo
          key: str, # Llave maestra del archivo
          mode: str = 'a', # Agrega lineas a un archivo existente (a) o crea una archivo nuevo (w)
          ...
         )
```

Por ejemplo

```python
>>> df.to_hdf("pandas_hdf.h5", key='excel', mode='w')
```

crea un archivo pandas_hdf.h5 en el directorio actual

Para lectura podemos usar la función `read_hdf`

```python
>>> mi_tabla_recuperada = pd.read_hdf("pandas_hdf.h5", key='/excel', mode='r')
```

OJO: Se requiere Pytables mayor a 3.5: https://github.com/PyTables/PyTables/issues/719

Podamos usar las librerías `PyTables` o `h5py` para trabajar con el archivo hdf5 en Python

```python
import h5py
with h5py.File("pandas_hdf.h5", mode="r") as f:
    print(f["excel"].keys())
    print(f["excel"]['block0_items'][()])
```